# Семинар 2

Сегодня мы с вами попробуем обучить модель для решения задачи абстрактивной суммаризации, при этом будем обучать модель в режиме половиной точности. Посмотрим как это делается с помощью удобной обертки из библиотеки pytorch-lightning и посмотрим как выглядит ее альтерантива на  голом Pytorch.  
После этого посмотрим, как можно глазами оценивать наиболее оптимальные чекпоинты и напишем код для параллельной SBS-оценки двух моделей. 

Код обучения я вынес в одну ячейку, чтобы при желании вы могли бы перенести его в скрипт и запускать из командной строки. В качестве датасета будем использовать датасет с мультиязычными суммаризациями новостных статей. Для нашего примера возьмем возьмем подмножество на русском языке, чтобы удобнее и быстрее его оценивать 

В данном фрагменте мы используем обертку Fabric из библиотеки Pytorch-Lightning(на ее примере в прошлом семинаре мы исследовали как обучаются модели в режиме разной точности), она позволяет удобно сделать поддержку разных режимов обучения снимая часть рутины с вас

In [4]:
import os
import sys
import torch
from lightning import Fabric
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel
from datasets import load_dataset
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")


class CustomDataset(Dataset):
    def __init__(self, examples, tokenizer, max_input_length=1024, max_target_length=128):
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        inputs = self.tokenizer(example["text"], 
                                max_length=self.max_input_length, padding="max_length", truncation=True, return_tensors="pt")
        targets = self.tokenizer(example["summary"], 
                                 max_length=self.max_target_length, padding="max_length", truncation=True, return_tensors="pt")
        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()
        labels = targets["input_ids"].squeeze()
        # we replace -100 in the labels as CrossEntropyLoss ignore_index value
        labels[labels == self.tokenizer.pad_token_id] = -100
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


# Preprocessing function
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(text=examples["summary"], max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


def train():
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/ {num_epochs}"):  
            model.train()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            
            fabric.backward(loss)
            optimizer.step()
            optimizer.zero_grad()
            
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")
    
        # Evaluation loop
        model.eval()
        total_eval_loss = 0
        with torch.no_grad():
            for batch in tqdm(valid_loader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                total_eval_loss += loss.item()
            print(f"Validation Loss: {total_eval_loss/len(valid_loader)}")
        fabric.save(f'./summ_models/ep{epoch + 1}_val_loss_{total_eval_loss/len(valid_loader):.3f}', model.state_dict())
    

# Load and preprocess the dataset
dataset = load_dataset("mlsum", "ru")
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruT5-base", use_fast=False, legacy=False,
                                          max_length=1024, padding="max_length", truncation=True, return_tensors="pt")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

prefix = "summarize: "
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Prepare the DataLoader
train_dataset = CustomDataset(tokenized_dataset["train"], tokenizer)
valid_dataset = CustomDataset(tokenized_dataset["validation"], tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16)

fabric = Fabric(accelerator="cuda", devices=1, precision="bf16-mixed")
fabric.launch()

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/ruT5-base")
model.cuda()

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

model, optimizer = fabric.setup(model, optimizer)

# Training loop
num_epochs = 10
model.train()

train()

fabric.print(f"Memory used: {torch.cuda.max_memory_reserved() / 1e9:.02f} GB")

Using bfloat16 Automatic Mixed Precision (AMP)
Epoch 1/ 10: 100%|██████████| 1598/1598 [11:24<00:00,  2.33it/s]


Epoch 1/10, Loss: 3.5164777173267887


100%|██████████| 47/47 [00:13<00:00,  3.57it/s]


Validation Loss: 2.7277334345147963


Epoch 2/ 10: 100%|██████████| 1598/1598 [11:24<00:00,  2.33it/s]


...

Validation Loss: 2.6417242009589015


Epoch 10/ 10: 100%|██████████| 1598/1598 [11:25<00:00,  2.33it/s]


Epoch 10/10, Loss: 2.376209890588801


100%|██████████| 47/47 [00:12<00:00,  3.65it/s]


Validation Loss: 2.674687979069162
Memory used: 67.54 GB


Мы обучили несколько чекпоинтов, и теперь можем выбирать, наиболее оптимальный чекпоинт на основе ручной валидации. Про автоматические метрики мы также не забываем, просто посчитаем для простоты их только для одного чекпонита 

Теперь давайте посмотрим, как будет выглядеть код обучения без сторонних библиотек - только чистый Pytorch)) 

In [1]:
import os
import sys
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel
from datasets import load_dataset
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")


class CustomDataset(Dataset):
    def __init__(self, examples, tokenizer, max_input_length=1024, max_target_length=128):
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        inputs = self.tokenizer(example["text"], 
                                max_length=self.max_input_length, padding="max_length", truncation=True, return_tensors="pt")
        targets = self.tokenizer(example["summary"], 
                                 max_length=self.max_target_length, padding="max_length", truncation=True, return_tensors="pt")
        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()
        labels = targets["input_ids"].squeeze()
        # Replace -100 in the labels as PyTorch's CrossEntropyLoss ignore_index
        labels[labels == self.tokenizer.pad_token_id] = -100
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [2]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(text=examples["summary"], max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Нам нужен будет код из функции  train, поэтому только  его и оставим

Обратите внимание, что мы добавляем импорт из модуля amp - [Automatic Mixid Precision](https://pytorch.org/tutorials/recipes/recipes/amp_recipe.html)

Scaler будет отвечать за то, чтобы при проведении вычислений с пониженной точностью у нас не было NaN значений, которые могут не поместится в диапазон поддерживаемых значений. Чтобы предотвратить потерю точности, Scaler умножает loss на коэффициент масштабирования и вызывает backward на масштабированных значениях loss`а. Градиенты, которые протекают в результате обратного обратного распространения ошибки, затем масштабируются тем же коэффициентом. Другими словами, значения градиентов имеют большую величину, так что они не обнуляются.

In [34]:
from torch.cuda.amp import GradScaler, autocast


def train(use_amp=True):
    scaler = GradScaler()
    
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/ {num_epochs}"):  
            model.train()
            with torch.autocast(device_type='cuda', dtype=torch.bfloat16, enabled=use_amp):  # MAIN DIFFERENCE IS HERE
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                total_loss += loss.item()
            
            scaler.scale(loss).backward()  # AND HERE
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")
    
        # Evaluation loop
        model.eval()
        total_eval_loss = 0
        with torch.autocast(device_type='cuda', dtype=torch.bfloat16, enabled=use_amp):  # AND HERE
            with torch.no_grad():
                for batch in tqdm(valid_loader):
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['labels'].to(device)
                    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    total_eval_loss += loss.item()
                print(f"Validation Loss: {total_eval_loss/len(valid_loader)}")
        torch.save(model.state_dict(), f'./summ_models/ep{epoch + 1}_val_loss_{total_eval_loss/len(valid_loader):.3f}.pt')  # ALSO HERE
        # if then you will need to resume traning do not forget to save optimizer and scaler!
        
        # torch.save(optimizer.state_dict(), f'./summ_models/optim_ep{epoch + 1}_val_loss_{total_eval_loss/len(valid_loader):.3f}.pt')
        # torch.save(scaler.state_dict(), f'./summ_models/scaler_ep{epoch + 1}_val_loss_{total_eval_loss/len(valid_loader):.3f}.pt')

In [3]:
# Load and preprocess the dataset
dataset = load_dataset("mlsum", "ru")
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruT5-base", use_fast=False, legacy=False,
                                          max_length=1024, padding="max_length", truncation=True, return_tensors="pt")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

prefix = "summarize: "
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Prepare the Datasets and DataLoaders
train_dataset = CustomDataset(tokenized_dataset["train"], tokenizer)
valid_dataset = CustomDataset(tokenized_dataset["validation"], tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16)

In [36]:
# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/ruT5-base")
model.cuda()

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
num_epochs = 10

train(use_amp=True)

print(f"Memory used: {torch.cuda.max_memory_reserved() / 1e9:.02f} GB")

Epoch 1/ 10: 100%|██████████| 1598/1598 [13:12<00:00,  2.02it/s]


Epoch 1/10, Loss: 3.5443742300601717


100%|██████████| 47/47 [00:12<00:00,  3.79it/s]


Validation Loss: 2.7520990118067314


Epoch 2/ 10:  47%|████▋     | 746/1598 [06:10<07:03,  2.01it/s]


KeyboardInterrupt: 

Так как современные декодерные модели достаточно велики, то очень часто мы ограничены в ресурсах. Чтобы увеличить эфективный размер батча для подсчета градиентов часто прибегают к хитрому трюку, когда мы накапливаем градиент за несколько шагов и только после определенного порога делаем обновление весов модели. Врядли вам часто придется писать это самостоятельно, однако реализацию этого действия на Pytorch мы сейчас посмотрим  

Такое накопление позволяет вам эффективнее работать с данными, однако это черевато возможным ростом величины ошибки, поэтому с этим параметром нужно быть аккуратными. Также обратите внимание, что здесь мы закомментировали операцию unscaling`а градументов, это делается для того чтобы лучше контролировать изменение весов модели. Здесь также есть тонкий момент, потому что scaler за вас нормирует ваши градиенты и выполняя unscale вам нужно самостоятельно проделать клиппинг норм градиентов или их абсолютные значения

In [4]:
from torch.cuda.amp import GradScaler, autocast


def train(use_amp=True, iters_to_accumulate=4):
    scaler = GradScaler()
    
    for epoch in range(num_epochs):
        total_loss = 0
        for i, batch in tqdm(enumerate(train_loader), desc=f"Epoch {epoch + 1}/ {num_epochs}", total=len(train_loader)):  
            model.train()
            with torch.autocast(device_type='cuda', dtype=torch.bfloat16, enabled=use_amp):  
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                total_loss += loss.item()
            
            scaler.scale(loss).backward()

            if (i + 1) % iters_to_accumulate == 0:  # MAIN DIFFERENCE IS HERE
                # may unscale_ here if desired (e.g., to allow clipping unscaled gradients)
                
                # Unscales the gradients of optimizer's assigned params in-place
                # scaler.unscale_(optimizer)
                # # Since the gradients of optimizer's assigned params are unscaled, clips as usual:
                # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")
    
        # Evaluation loop
        model.eval()
        total_eval_loss = 0
        with torch.autocast(device_type='cuda', dtype=torch.bfloat16, enabled=use_amp): 
            with torch.no_grad():
                for batch in tqdm(valid_loader):
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['labels'].to(device)
                    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    total_eval_loss += loss.item()
                print(f"Validation Loss: {total_eval_loss/len(valid_loader)}")
        torch.save(model.state_dict(), f'./summ_models/ep{epoch + 1}_val_loss_{total_eval_loss/len(valid_loader):.3f}.pt')
        # if then you will need to resume traning do not forget to save optimizer and scaler!
        
        # torch.save(optimizer.state_dict(), f'./summ_models/optim_ep{epoch + 1}_val_loss_{total_eval_loss/len(valid_loader):.3f}.pt')
        # torch.save(scaler.state_dict(), f'./summ_models/scaler_ep{epoch + 1}_val_loss_{total_eval_loss/len(valid_loader):.3f}.pt')

In [5]:
model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/ruT5-base")
model.cuda()

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
num_epochs = 10

train(use_amp=True, iters_to_accumulate=4)

print(f"Memory used: {torch.cuda.max_memory_reserved() / 1e9:.02f} GB")

Epoch 1/ 10: 100%|██████████| 1598/1598 [11:37<00:00,  2.29it/s]


Epoch 1/10, Loss: 3.8105912393562784


100%|██████████| 47/47 [00:12<00:00,  3.78it/s]


Validation Loss: 2.7997365099318485


Epoch 2/ 10:  71%|███████   | 1135/1598 [08:16<03:22,  2.29it/s]


KeyboardInterrupt: 

Как видите обучение теперь проходит немного быстрее, однако цена этого ускорения может быть оценена только с использованием конвеера метрик, который вы используете

## Посчитаем автоматические метрики

Как рассатривали на лекции, нашей основной автоматической метрикой качества для задачи суммаризации будет метрика ROUGE. Посмотрим сразу на несколько ее разновидностей - ROUGE-1 для униграмм, ROUGE-2 для биграм и ROUGE-L для наибольшей общей последовательности слов(они могут идти не подряд)

In [6]:
from rouge import Rouge

def get_rouge_scores(actual_summary, predicted_summary):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, actual_summary)
    return [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']]

In [7]:
def predict_summary(document, model, tokenizer):
  device = model.device
  model.eval()
  with torch.no_grad():
      tokenized = tokenizer(["summarize:" + document], truncation=True, padding ='longest',return_tensors='pt')
      tokenized = {k: v.to(device) for k, v in tokenized.items()}
      tokenized_result = model.generate(**tokenized, max_length=128)
      tokenized_result = tokenized_result.detach().to('cpu')
  predicted_summary = tokenizer.decode(tokenized_result[0])
  torch.cuda.empty_cache()
  return predicted_summary

In [8]:
# rouge score of validation data

rouge1_scores = []
rouge2_scores = []
rougel_scores = []

pred_summary_list = []

val_scores = dict()

for i in tqdm(range(len(dataset['test']))):
  doc = dataset['test'][i]['text']
  if len(doc) > 10000:  # skip very long texts
      continue
  pred_summary = predict_summary(doc, model, tokenizer)
  human_summary = dataset['test'][i]['summary']

  score = get_rouge_scores(human_summary, pred_summary)

  rouge1_scores.append(score[0])
  rouge2_scores.append(score[1])
  rougel_scores.append(score[2])

  pred_summary_list.append(pred_summary)

val_scores["pred_summary"] = pred_summary_list

val_scores['rouge1'] = rouge1_scores
val_scores['rouge2'] = rouge2_scores
val_scores['rougel'] = rougel_scores

val_scores

100%|██████████| 757/757 [02:09<00:00,  5.86it/s]


{'pred_summary': ['<pad> В России скоро появится интернет по всему миру бесплатно</s>',
  '<pad> В КНДР не удалось договориться о ядерном разоружении</s>',
  '<pad> В Подмосковье ввели в эксплуатацию мобильное приложение «Скорая помощь»</s>',
  '<pad> В честь юбилея группы «Ленинград» в сети появился клип на песню «Касайся»</s>',
  '<pad> В Раменском рассказали о том, как убили жену бывшего главы района</s>',
  '<pad> В Подмосковье откроются 96 оздоровительных лагерей и 1124 с дневным пребыванием</s>',
  '<pad> «МК» узнал, что произошло с продюсером «Мария, мать Иисуса»</s>',
  '<pad> В июне рубль может упасть на 6,6%</s>',
  '<pad> В аэропорту Пулково начали строить современный топливозаправочный комплекс</s>',
  '<pad> Президент США Дональд Трамп настрочил пару ласковых в адрес мэра Лондона</s>',
  '<pad> В Иерусалиме пройдет встреча советника президента США по национальной безопасности Джона Болтона и секретаря Совбеза РФ Николая Патрушева</s>',
  '<pad> «МК» узнал, как будет проход

In [9]:
# average rouge 1
torch.Tensor(val_scores['rouge1']).mean()

tensor(0.0629)

In [10]:
# average rouge 2
torch.Tensor(val_scores['rouge2']).mean()

tensor(0.0082)

In [11]:
# average rouge 2
torch.Tensor(val_scores['rougel']).mean()

tensor(0.0583)

# Сравним две модели

Финальной метрикой качества для таких субъективных задач как суммаризация в любом случае остается оценка человеком, и даже если ваша автометрика отлично коррелирует с человеческими оценками, беглую проверку глазами в любом случае стоит делать

Мы реализуем простой интерфейс для параллельных сравнений генераций 2 моделей, с интерактивным интерфейсом, но при желании вы можете прогнать генерации на отложенной выборке и также сравнить их друг против друга

In [1]:
import os
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
from typing import Tuple, List

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel
from ipywidgets import HBox, Label, VBox, Button, Layout, Text, Textarea, HTML, Output
from IPython.display import display

In [2]:
TOKENIZER_PATH = "ai-forever/ruT5-base"
MODEL_NAME = "ai-forever/ruT5-base"
MODEL1_STATE = './summ_models/ep8_val_loss_2.652.pt'
MODEL2_STATE = './summ_models/ep9_val_loss_2.642.pt'

Загрузим 2 модели и проинициализируем их из двух последних чекпоинтов

In [3]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruT5-base", use_fast=False, legacy=False,
                                          max_length=1024, padding="max_length", truncation=True, return_tensors="pt")
model1 = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/ruT5-base", torch_dtype=torch.bfloat16)
model2 = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/ruT5-base", torch_dtype=torch.bfloat16)

model1.load_state_dict(torch.load(MODEL1_STATE))
model2.load_state_dict(torch.load(MODEL2_STATE))

<All keys matched successfully>

In [4]:
from datasets import load_dataset

dataset = load_dataset("mlsum", "ru")

In [28]:
GENERATION_PARAMS = {
    "do_sample": True,
    "temperature": 0.8,
    "top_p": 0.95,
    "top_k": 30, 
    "max_length": 128,
    "repetition_penalty": 1.07,
}


def get_models_answers(
    model_left,
    model_right,
    inp_text: str,
    generation_params: dict,
) -> Tuple[str, str, str]:
    inp_text = f"summarize:{inp_text}"

    with torch.no_grad():
      tokenized = tokenizer(["summarize:" + inp_text], truncation=True, padding ='longest',return_tensors='pt')
      tokenized = {k: v.to(device) for k, v in tokenized.items()}
      tokenized_result_l = model_left.generate(**tokenized, max_length=128)
      tokenized_result_l = tokenized_result_l.detach().to('cpu')
      predicted_summary_l = tokenizer.decode(tokenized_result_l[0])
    
    del tokenized_result_l
    torch.cuda.empty_cache()

    with torch.no_grad():
      tokenized = tokenizer(["summarize:" + inp_text], truncation=True, padding ='longest',return_tensors='pt')
      tokenized = {k: v.to(device) for k, v in tokenized.items()}
      tokenized_result_r = model_right.generate(**tokenized, max_length=128)
      tokenized_result_r = tokenized_result_r.detach().to('cpu')
      predicted_summary_r = tokenizer.decode(tokenized_result_r[0])
    
    del tokenized_result_r
    del tokenized
    torch.cuda.empty_cache()
    return inp_text, predicted_summary_l, predicted_summary_r


def generate_answers(model1, model2, inp_texts: List[str], generation_params):
    with ThreadPoolExecutor(1) as executor:
        prev_future = executor.submit(
            get_models_answers,
            model1,
            model2,
            inp_texts[0],  # Corrected to use the first item explicitly
            generation_params,
        )
        for inp_text in inp_texts[1:]:
            cur_future = executor.submit(
                get_models_answers,
                model1,
                model2,
                inp_text,
                generation_params,
            )
            yield prev_future.result()  # Yields the result of the previous future
            prev_future = cur_future  # Updates prev_future to the current future
        
        yield prev_future.result()  # Yields the result of the last future after the loop



def display_choice(texts: Tuple[str, str, str]):
    prompt, left_text, right_text = texts
    box = VBox(
        [
            Textarea(prompt, height="100%",
                  width="auto",
                  rows=10, layout=Layout(width="auto")),
            HBox(
                [
                    HTML(
                        value='<style> .left { background-color: LightGreen; width: 500px} </style> <div class="left">'
                        + left_text
                        + "</div>"
                    ),
                    HTML(
                        value='<style> .right { background-color: LightSkyBlue; width: 500px} </style> <div class="right">'
                        + right_text
                        + "</div>"
                    ),
                ]
            ),
            
        ]
    )

    return box
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
options = ["left better", "right better", "both good", "both shit", "skip"]
click_counts = {opt: 0 for opt in options}
text_iter = generate_answers(model1.to(device), model2.to(device), dataset['test'][:1]['text'], GENERATION_PARAMS)

text_display_area = VBox([HTML("Model outputs will appear here")])

# Initialize the Output widget for dynamic updates
output_area = Output()

# Define a function to handle button clicks
def on_button_click(b):
    # Check if 'b' is not None before trying to access its attributes
    if b is not None:
        click_counts[b.description] += 1

    with output_area:
        output_area.clear_output()
        texts = next(text_iter, None)
        if texts:
            display(display_choice(texts))
        else:
            print("No more texts to display")

    # Optionally display or log the click counts
    # This can be outside the 'if b is not None' check if you want to log on manual triggers as well
    if b is not None:
        print("Click counts:", click_counts)


# Create buttons and assign the click event
buttons = [Button(description=opt) for opt in options]
for button in buttons:
    button.on_click(on_button_click)

# Display everything
display(HBox(buttons))
display(output_area)

# Call the updater function to display the first item
on_button_click(None)  # Manually trigger the display update for the first item

Output()

In [29]:
Counter(click_counts)

Counter({'both good': 1,
         'left better': 0,
         'right better': 0,
         'both shit': 0,
         'skip': 0})